# Task I - Using linear regression for forecasting

### 1.Data preparation: Use one of the methods described in Project 1 (Advanced), Task 1 to pre-process the trace. Remove possible outliers. Reduce the dimensionality of the feature space to k = 16 using tree-based feature selection. Then, split the processed trace into training and test samples (x(t),y(t)) by assigning the samples with t < T to the training set and t ≥T to the test set. T is chosen so that the training set contains 70% of the samples.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')
X.index = pd.to_datetime(X['TimeStamp'])
Y.index = pd.to_datetime(Y['TimeStamp'])
X_dropped = X.drop(labels=["Unnamed: 0", "TimeStamp"], axis=1, inplace=False)
Y_dropped = Y.drop(labels=["Unnamed: 0", "TimeStamp"], axis=1, inplace=False)
X_preprocessed = pd.DataFrame()
X_tmp = preprocessing.StandardScaler().fit_transform(X_dropped)
for i, n in enumerate(X_dropped):
        X_preprocessed[n] = X_tmp[:, i]
        
X_preprocessed.head()



/home/anisha/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/anisha/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,0_all_..usr,0_all_..sys,0_all_..iowait,0_all_..soft,0_all_..idle,0_cpu0_.usr,0_cpu0_.sys,0_cpu0_.iowait,0_cpu0_.soft,0_cpu0_.idle,...,36_RxBytes.1,36_TxBytes.1,40_RxPacktes.1,40_TxPacktes.1,40_RxBytes.1,40_TxBytes.1,41_RxPacktes.1,41_TxPacktes.1,41_RxBytes.1,41_TxBytes.1
0,0.069116,1.088334,0.766004,0.573967,-0.579449,1.364018,0.977205,-0.00831,-0.355042,-2.078731,...,1.583733,1.335871,1.090350,1.069987,1.211176,0.975349,1.053685,1.064968,0.971475,1.180097
1,0.247507,2.503497,0.377145,0.573967,-0.865816,1.364018,4.193380,-0.00831,-0.355042,-2.078731,...,1.551250,1.485179,1.190029,1.263907,1.110672,1.287438,1.269613,1.224345,1.294599,1.157325
2,0.158311,1.421314,0.571574,-0.290273,-0.579449,1.384053,0.993448,-0.00831,-0.355042,-1.518028,...,1.636638,1.320954,1.119925,1.012137,1.234329,0.859493,1.037388,1.111521,0.901761,1.236388
3,0.425898,1.421314,0.766004,3.382748,-1.135338,1.404089,-0.630882,-0.00831,2.626215,-0.945519,...,1.503362,1.510045,1.144023,1.126207,1.215636,1.070883,1.113167,1.141096,1.054886,1.212100
4,0.158311,0.006151,5.432315,1.654267,-2.129200,2.355777,2.585292,-0.00831,-0.355042,-2.078731,...,1.556992,1.341298,1.134713,1.095245,1.194114,1.050206,1.113574,1.146025,1.094834,1.186764


In [4]:
X_preprocessed.shape

(14481, 1751)

In [5]:
#converting Y_dropped to make it of the same form as X_preprocessed
Y_new = pd.DataFrame()
Y_tmp=Y_dropped.to_numpy()

for i, n in enumerate(Y_dropped):
        Y_new[n] = Y_tmp[:, i]  
# outlier rejection   
remove = []
for i in X_preprocessed:
    for j in range(len(X_preprocessed[i])):
        if j not in remove and abs(X_preprocessed[i][j]) > 80:
            remove.append(j)
X_clean = X_preprocessed.drop(labels=remove, axis=0, inplace=False)
Y_clean = Y_new.drop(labels=remove, axis=0, inplace=False)

print("Number of dropped samples: ", (len(remove)))
%store X_clean
%store Y_clean

Number of dropped samples:  136
Stored 'X_clean' (DataFrame)
Stored 'Y_clean' (DataFrame)


In [14]:
Y_clean.shape

(14345, 2)

In [70]:
#time index sorting
X_clean = X_latest.reset_index()
Y_clean = Y_latest.reset_index()
X_clean = X_clean.drop(labels= "index", axis=1)
Y_clean = Y_clean.drop(labels= "index", axis=1)
X_clean

,feature1 : 1_i127_intr.s,feature2 : 5_campg.s,feature3 : 4_RxPacktes,feature4 : 4_TxPacktes,feature5 : 4_TxBytes,feature6 : 15_RxPacktes,feature7 : 15_TxBytes,feature8 : 29_TxPacktes,feature9 : 29_TxBytes,feature10 : 40_TxPacktes,feature11 : 41_TxPacktes,feature12 : 4_TxBytes.1,feature13 : 29_TxBytes.1,feature14 : 30_RxBytes.1,feature15 : 40_RxPacktes.1,feature16 : 40_TxPacktes.1
0,1.119895,0.019228,1.039869,1.088463,1.214220,1.028539,1.184277,1.078469,1.165585,1.069987,1.064968,1.214220,1.165585,1.122090,1.090350,1.069987
1,0.855128,0.051963,1.299455,1.238908,1.194544,1.247687,1.219757,1.202829,1.201460,1.263907,1.224345,1.194544,1.201460,1.194057,1.190029,1.263907
2,0.879198,0.043744,1.025629,1.115817,1.240098,1.079864,1.105230,1.110792,1.237204,1.012137,1.111521,1.240098,1.237204,1.230830,1.119925,1.012137
3,0.566292,0.022571,1.138740,1.127852,1.187528,1.132818,1.172840,1.183655,1.189385,1.126207,1.141096,1.187528,1.189385,1.193328,1.144023,1.126207
4,0.939372,0.048202,1.112293,0.980143,1.005623,1.148704,1.167679,1.119557,1.219838,1.095245,1.146025,1.005623,1.219838,1.225559,1.134713,1.095245
5,0.746815,0.013935,1.063468,1.166147,1.318807,1.068458,1.253234,1.137088,1.167555,1.061839,1.114260,1.318807,1.167555,1.187147,1.114448,1.061839
6,1.143965,0.043465,1.235169,1.173259,1.119624,1.169071,1.173450,1.191325,1.248986,1.229686,1.206819,1.119624,1.248986,1.231431,1.208103,1.229686
7,1.083791,0.011984,1.087881,1.080804,1.169513,1.086381,1.216996,1.126679,1.172659,1.095653,1.119189,1.169513,1.172659,1.179884,1.118830,1.095653
8,1.011581,0.048062,1.210757,1.138794,1.143921,1.136891,1.187042,1.142567,1.215889,1.128652,1.057300,1.143921,1.215889,1.215321,1.129236,1.128652
9,0.939372,0.047366,1.134671,1.183106,1.198475,1.134447,1.204705,1.171054,1.163403,1.150651,1.128499,1.198475,1.163403,1.121947,1.096922,1.150651


In [71]:
%store -r X_clean
%store -r Y_clean

#Reduce the dimensionality of the feature space 
tree = ExtraTreesRegressor(n_estimators=100, n_jobs=20)
tree = tree.fit(X_clean, Y_clean)
model = SelectFromModel(tree,prefit=True,max_features=16,threshold=-np.inf)
X_fs = model.transform(X_clean)
top_features = []
get_feat = model.get_support()
for i, n in enumerate(X_clean):
    if get_feat[i]:
        top_features.append(n)
feature_name = []
for i in range(16):
    feature_name.append('feature'+str(i+1)+" : "+top_features[i])
X_latest = pd.DataFrame(data = X_fs,columns= feature_name)
Y_latest = pd.DataFrame(data = Y_clean,columns= Y_dropped.columns)
X_latest.head()
Y_latest.head()

,ReadsAvg,WritesAvg
0,59.1331,118.7723
1,59.8588,118.4950
2,57.8251,116.8042
3,63.8154,128.3462
4,57.4993,118.2260


In [72]:
X_latest

,feature1 : 1_i127_intr.s,feature2 : 4_TxBytes,feature3 : 15_TxPacktes,feature4 : 15_TxBytes,feature5 : 17_TxPacktes,feature6 : 30_RxPacktes,feature7 : 40_RxBytes,feature8 : 41_TxPacktes,feature9 : 4_RxPacktes.1,feature10 : 4_TxPacktes.1,feature11 : 4_TxBytes.1,feature12 : 15_RxPacktes.1,feature13 : 15_TxBytes.1,feature14 : 17_TxPacktes.1,feature15 : 29_TxBytes.1,feature16 : 40_RxBytes.1
0,1.119895,1.214220,1.035940,1.184277,1.034312,1.056585,1.211176,1.064968,1.039869,1.088463,1.214220,1.028539,1.184277,1.034312,1.165585,1.211176
1,0.855128,1.194544,1.290084,1.219757,1.238812,1.201219,1.110672,1.224345,1.299455,1.238908,1.194544,1.247687,1.219757,1.238812,1.201460,1.110672
2,0.879198,1.240098,0.999243,1.105230,1.077493,1.111918,1.234329,1.111521,1.025629,1.115817,1.240098,1.079864,1.105230,1.077493,1.237204,1.234329
3,0.566292,1.187528,1.110978,1.172840,1.135340,1.174922,1.215636,1.141096,1.138740,1.127852,1.187528,1.132818,1.172840,1.135340,1.189385,1.215636
4,0.939372,1.005623,1.138365,1.167679,1.125563,1.125067,1.194114,1.146025,1.112293,0.980143,1.005623,1.148704,1.167679,1.125563,1.219838,1.194114
5,0.746815,1.318807,1.108240,1.253234,1.048163,1.154651,1.254974,1.114260,1.063468,1.166147,1.318807,1.068458,1.253234,1.048163,1.167555,1.254974
6,1.143965,1.119624,1.209569,1.173450,1.205407,1.171635,1.179406,1.206819,1.235169,1.173259,1.119624,1.169071,1.173450,1.205407,1.248986,1.179406
7,1.083791,1.169513,1.114812,1.216996,1.055495,1.133833,1.211234,1.119189,1.087881,1.080804,1.169513,1.086381,1.216996,1.055495,1.172659,1.211234
8,1.011581,1.143921,1.167394,1.187042,1.174447,1.138763,1.157781,1.057300,1.210757,1.138794,1.143921,1.136891,1.187042,1.174447,1.215889,1.157781
9,0.939372,1.198475,1.200257,1.204705,1.118638,1.139311,1.048923,1.128499,1.134671,1.183106,1.198475,1.134447,1.204705,1.118638,1.163403,1.048923


In [73]:
#time index sorting
X_clean = X_latest.sort_index(axis = 0)
Y_clean = Y_latest.sort_index(axis = 0)

In [74]:
#splitting the processed trace into training and test samples
X_train, X_test, Y_train, Y_test = train_test_split(X_latest, Y_latest, test_size=0.3, shuffle = False)
print(X_train.shape,"(70% of the samples in training set and 16 features)")

(10041, 16) (70% of the samples in training set and 16 features)


In [75]:
print(X_train.head())
print(Y_train.head())
print(X_test.head())
print(Y_test.head())

   feature1 : 1_i127_intr.s  feature2 : 4_TxBytes  feature3 : 15_TxPacktes  \
0                  1.119895              1.214220                 1.035940   
1                  0.855128              1.194544                 1.290084   
2                  0.879198              1.240098                 0.999243   
3                  0.566292              1.187528                 1.110978   
4                  0.939372              1.005623                 1.138365   

   feature4 : 15_TxBytes  feature5 : 17_TxPacktes  feature6 : 30_RxPacktes  \
0               1.184277                 1.034312                 1.056585   
1               1.219757                 1.238812                 1.201219   
2               1.105230                 1.077493                 1.111918   
3               1.172840                 1.135340                 1.174922   
4               1.167679                 1.125563                 1.125067   

   feature7 : 40_RxBytes  feature8 : 41_TxPacktes  feature9 : 